In [1]:
import os
import time
import boto3
import json


# Initialize AWS clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=region)


account_id_suffix = account_id[:3]
region_suffix = ''.join([word[0] for word in region.split('-')])
suffix = f"{region_suffix}-{account_id_suffix}"

# Define Knowledge Base parameters
knowledge_base_name_standard = 'fixed-size-chunk-kb'
knowledge_base_name_hierarchical = 'hierarchical-chunk-kb'
knowledge_base_description = "Knowledge Base containing sample synthetic Octank financial data"

print(f"AWS Region: {region}", f"Account ID: {account_id}", f"Suffix: {suffix}", sep='\n')

AWS Region: us-east-1
Account ID: 533267284022
Suffix: ue1-533


In [8]:
from utils.knowledgebase import BedrockKnowledgeBases
from utils.agents import BedrockAgents
from utils.agents import AWSResourceManager
from utils.chat import BedrockChat, SyntheticDataGenerator
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

    
kb = BedrockKnowledgeBases()
agents = BedrockAgents()

In [9]:
agent_name=f'rag-agent-{suffix}'
kb_id = "892DWWRLXW"

print(f"Agent name: {agent_name}", f"KB id: {kb_id}", sep='\n')


Agent name: rag-agent-ue1-533
KB id: 892DWWRLXW


In [10]:
user_query = "What is the total cost of Octank Financial's property and equipment as of December 31, 2022?"

result = agents.invoke(
    agent_name=agent_name, 
    input_text=user_query, 
    verbose=True, 
    trace_level='core',
    save_trace_json_file='agent_full_trace.json'
)


2025-01-07 16:02:46,107 [INFO] Invoking agent 'rag-agent-ue1-533' with input: What is the total cost of Octank Financial's property and equipment as of December 31, 2022?
---- Step 1 ----
Took 3.7s, using 3251 tokens (in: 3145, out: 106) to complete action.
Final response:
According to the table for Property and Equipment, Net under NOTES TO CONSOLIDATED FINANCIAL STATEMENTS, the total cost of Octank Financial's property and equipment as of December 31, 2022 is $440,000,000....
Agent made a total of 1 LLM calls, using 3251 tokens (in: 3145, out: 106), and took 4.2 total seconds.


In [11]:
print_markdown(result['response'])

According to the table for Property and Equipment, Net under NOTES TO CONSOLIDATED FINANCIAL STATEMENTS, the total cost of Octank Financial's property and equipment as of December 31, 2022 is $440,000,000.

In [17]:
# Basic approach to read the stream
def display_stream(event_stream):
    for event in event_stream:
        if 'trace' in event:
            trace = event['trace']
            print(trace)
            
# More detailed approach with formatting
def display_stream_detailed(event_stream):
    for event in event_stream:
        if 'trace' in event:
            trace = event['trace']
            print("-" * 50)
            print(f"Type: {trace.get('type', 'N/A')}")
            print(f"Content: {trace.get('content', 'N/A')}")
            print(f"Timestamp: {trace.get('timestamp', 'N/A')}")

# If you want to collect events while displaying
def collect_and_display_stream(event_stream):
    events = []
    for event in event_stream:
        if 'trace' in event:
            trace = event['trace']
            print(f"Received event: {trace.get('type', 'N/A')}")
            events.append(trace)
    return events

# Using with async/await (if the stream is asynchronous)
async def display_async_stream(event_stream):
    async for event in event_stream:
        if 'trace' in event:
            trace = event['trace']
            print(trace)


In [21]:
display_async_stream(result['trace'])

<coroutine object display_async_stream at 0x117768190>

In [22]:
result['trace']

In [23]:
def process_bedrock_stream(event_stream):
    final_response = None
    all_traces = []
    
    try:
        for event in event_stream:
            if 'chunk' in event:
                # Process response chunks
                data = event['chunk']['bytes']
                final_response = data.decode('utf-8')
                print("\nResponse chunk:", final_response)
                
            elif 'trace' in event:
                # Process trace events
                trace_data = event['trace']
                all_traces.append(trace_data)
                print("\nTrace event:", json.dumps(trace_data, indent=2))
                
            else:
                print("\nUnknown event type:", event)
                
        return {
            "final_response": final_response,
            "traces": all_traces
        }
        
    except Exception as e:
        print(f"Error processing stream: {str(e)}")
        return None

# Using in a notebook
from IPython.display import JSON, display

def display_stream_results(event_stream):
    results = process_bedrock_stream(event_stream)
    
    if results:
        print("\n=== Final Response ===")
        print(results["final_response"])
        
        print("\n=== All Traces ===")
        display(JSON(results["traces"]))

# Use it with your event stream
event_stream = result['trace']
display_stream_results(event_stream)



=== Final Response ===
None

=== All Traces ===


<IPython.core.display.JSON object>